In [23]:
import numpy as np
import torch
import os
from itertools import repeat

import h5py
import pickle
import fnmatch
import cv2
import sys
from time import time
from torch.utils.data import TensorDataset, DataLoader
from scipy.spatial.transform import Rotation

np.set_printoptions(suppress=True)


In [2]:
act_path = "/home/zfei/code/act-plus-plus/"
sys.path.append(act_path)
from utils import EpisodicDataset
from utils import find_all_hdf5, flatten_list, get_norm_stats

%load_ext autoreload
%autoreload 2

In [3]:
dataset_path_list = find_all_hdf5("/mnt/data1/act/train_act_scara_3cam/data1/", skip_mirrored_data=False)
norm_stats, all_episode_len = get_norm_stats(dataset_path_list)

camera_names = ['wrist', 'wrist_down', 'top']
train_episode_ids = [0, 1]
train_episode_len = [all_episode_len[i] for i in train_episode_ids]

chunk_size = 64
policy_class = "EPACT"

Found 51 hdf5 files


In [5]:
train_dataset = EpisodicDataset(dataset_path_list, 
                                camera_names, 
                                norm_stats, 
                                train_episode_ids, 
                                train_episode_len, 
                                chunk_size, 
                                policy_class)


URDF Path:  /home/zfei/code/act-plus-plus/urdf/hitbot_model.urdf
kdl_parse urdf ok?:  True
augment_images:  False
Initializing transformations


### Dataset for RS435i UPI

In [12]:
from datasets import EpisodicDatasetRs435i, get_norm_stats_folder, find_all_datafolder, load_folderdata

In [8]:
dataset_path_list = find_all_datafolder("/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/")
norm_stats, all_episode_len  = get_norm_stats_folder(dataset_path_list)

camera_names = ['color']
train_episode_ids = [0, 1]
train_episode_len = [all_episode_len[i] for i in train_episode_ids]

chunk_size = 64
policy_class = "ACTEP"
policy_class = "ACT"



Found 41 datafolders under /home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/


In [9]:
dataset = EpisodicDatasetRs435i(dataset_path_list, camera_names, norm_stats, train_episode_ids, train_episode_len, chunk_size, policy_class)

augment_images:  False
episode_id:  0
start_ts:  0
Initializing transformations


In [10]:
image_data, action_data, is_pad = dataset[50]

episode_id:  0
start_ts:  50


In [11]:
print(image_data.shape)
print(action_data.shape)
print(is_pad)

torch.Size([1, 3, 480, 848])
torch.Size([64, 7])
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True])


In [13]:
dataset_dir = "/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/"
name_filter = lambda n: True
batch_size_train = 8
batch_size_val = 8
chunk_size = 64
skip_mirrored_data = False
load_pretrain = False
stats_dir = None
sample_weights = None
train_ratio = 0.99
resize = None
seed = 0
policy_class = "ACTEP"

In [27]:
train_dataloader, val_dataloader, norm_stats, is_sim = load_folderdata(dataset_dir, 
                name_filter, 
                camera_names, 
                batch_size_train, 
                batch_size_val, 
                chunk_size, 
                skip_mirrored_data, 
                load_pretrain, 
                policy_class, 
                stats_dir_l=stats_dir, 
                sample_weights=sample_weights, 
                train_ratio=train_ratio, 
                resize=resize, 
                seed=seed)

Found 41 datafolders under /home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/
dataset_path_list_list:  [['/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/seg_00000', '/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/seg_00037', '/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/seg_00014', '/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/seg_00011', '/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/seg_00008', '/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/seg_00016', '/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/seg_00001', '/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/seg_00032', '/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/seg_00029', '/home/zfei/data/UPI/lab_sb/rs435i_lab

In [28]:
def repeater(data_loader):
    epoch = 0
    for loader in repeat(data_loader):
        for data in loader:
            yield data
        print(f'Epoch {epoch} done')
        epoch += 1


In [29]:
train_dataloader_rep = repeater(train_dataloader)


In [35]:
image_data, action_data, is_pad = next(train_dataloader_rep)

In [37]:
action_data.shape

torch.Size([8, 64, 7])